In [ ]:
#new method 

In [4]:
import os
# Force TensorFlow to run on CPU only and disable XLA devices
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_XLA_FLAGS"] = "--tf_xla_enable_xla_devices=false"

import numpy as np
import pandas as pd
import tensorflow as tf
import gpflow as gpf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from gmr import GMM  # Ensure you have installed the gmr package
from scipy.stats import multivariate_normal

# -------------------------Metrics-------------------------------------#
# Here NLPD is defined as the (positive) negative log predictive density,
# so smaller values are better (same convention as in Model 2).
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true))).numpy()

def ece(y_true, y_pred, n_bins=10):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    bin_edges = np.linspace(0, 1, n_bins + 1)
    bin_indices = np.digitize(y_pred, bin_edges, right=True) - 1
    ece_score = 0.0
    for i in range(n_bins):
        bin_mask = bin_indices == i
        if np.any(bin_mask):
            bin_true = y_true[bin_mask]
            bin_pred = y_pred[bin_mask]
            bin_acc = np.mean(bin_true)
            bin_conf = np.mean(bin_pred)
            ece_score += np.abs(bin_acc - bin_conf) * len(bin_true) / len(y_true)
    return ece_score



def nlpd(y_true, mu, sigma):
    return -np.mean(0.5 * np.log(2 * np.pi * sigma**2) + 0.5 * ((y_true - mu) / sigma)**2)

# -------------------------GMR Prediction Function-------------------------------------#
def predict_with_covariance(gmm, in_idx, X, epsilon=1e-6):
    """
    For each sample in X (shape [n_samples, n_features]),
    compute the conditional mean and variance for the output,
    which is assumed to be at index len(in_idx) in the joint data.
    """
    n_samples = X.shape[0]
    n_components = gmm.n_components
    out_idx = [len(in_idx)]  # output is at position equal to number of input features

    pred_means = np.zeros(n_samples)
    pred_vars = np.zeros(n_samples)

    for i in range(n_samples):
        x = X[i]  # Input vector for sample i
        comp_means = np.zeros(n_components)  # conditional mean per component
        comp_vars = np.zeros(n_components)   # conditional variance per component
        likelihoods = np.zeros(n_components) # likelihood for each component

        for k in range(n_components):
            mu = gmm.means[k]          # full mean for component k
            Sigma = gmm.covariances[k] # full covariance for component k

            # Partition into input and output parts:
            mu_x = mu[in_idx]
            mu_y = mu[out_idx][0]  # scalar output mean for component k
            Sigma_xx = Sigma[np.ix_(in_idx, in_idx)]
            Sigma_xy = Sigma[np.ix_(in_idx, out_idx)]
            Sigma_yx = Sigma[np.ix_(out_idx, in_idx)]
            Sigma_yy = Sigma[np.ix_(out_idx, out_idx)][0, 0]

            # Regularize Sigma_xx and compute its inverse
            Sigma_xx_reg = Sigma_xx + epsilon * np.eye(Sigma_xx.shape[0])
            inv_Sigma_xx = np.linalg.inv(Sigma_xx_reg)

            # Conditional mean and variance formulas:
            cond_mean = mu_y + Sigma_yx.dot(inv_Sigma_xx).dot(x - mu_x)
            cond_var = Sigma_yy - Sigma_yx.dot(inv_Sigma_xx).dot(Sigma_xy)
            # Set a per-component variance floor to avoid overconfident predictions.
            cond_var = max(cond_var, 0)

            comp_means[k] = cond_mean
            comp_vars[k] = cond_var

            # Retrieve component weight
            if hasattr(gmm, 'weights'):
                weight_k = gmm.weights[k]
            elif hasattr(gmm, 'priors'):
                weight_k = gmm.priors[k]
            else:
                raise AttributeError("GMM object has no attribute 'weights' or 'priors'.")
                
            likelihoods[k] = weight_k * multivariate_normal.pdf(x, mean=mu_x, cov=Sigma_xx_reg)

        total_likelihood = np.sum(likelihoods)
        if total_likelihood == 0:
            resp = np.ones(n_components) / n_components
        else:
            resp = likelihoods / total_likelihood

        overall_mean = np.sum(resp * comp_means)
        overall_var = np.sum(resp * (comp_vars + (comp_means - overall_mean)**2))

        pred_means[i] = overall_mean
        pred_vars[i] = overall_var
    return pred_means, pred_vars

# -------------------------Define RFF Layer-------------------------------------#
class RFFLayer(tf.keras.layers.Layer):
    def __init__(self, S, kernel_widths, feature_count, random_state=42, **kwargs):
        super(RFFLayer, self).__init__(**kwargs)
        self.S = S
        self.feature_count = feature_count
        self.kernel_widths = tf.constant(kernel_widths, dtype=tf.float32)
        np.random.seed(random_state)
        self.Z = [tf.constant(np.random.normal(0, 1, (S, 1)), dtype=tf.float32)
                  for _ in range(feature_count)]
        self.c = tf.constant(np.random.uniform(0, 2 * np.pi, S), dtype=tf.float32)

    def call(self, inputs):
        transformed_features = []
        for i in range(self.feature_count):
            feature = tf.expand_dims(inputs[:, i], axis=1)
            prod = feature * tf.transpose(self.Z[i])
            prod = prod / self.kernel_widths[i]
            prod = prod + self.c
            transformed = tf.sqrt(2.0 / self.S) * tf.cos(prod)
            transformed_features.append(transformed)
        bias = tf.ones((tf.shape(inputs)[0], 1))
        return tf.concat([bias] + transformed_features, axis=1)

# -------------------------Load Dataset-------------------------------------#
# For this example, we use the Auto MPG dataset.
data = pd.read_csv("/home/visdom/Downloads/auto-mpg.csv")
data['horsepower'] = pd.to_numeric(data['horsepower'], errors='coerce')
data = data.dropna(subset=['horsepower'])
X = data.drop(columns=['mpg']).copy()
y = data['mpg'].values
categorical_columns = X.select_dtypes(include=['object']).columns
if len(categorical_columns) > 0:
    X = pd.get_dummies(X, columns=categorical_columns)
X = X.values

# Set GPFlow default float type
gpf.config.set_default_float(tf.float32)

# -------------------------Experiment Loop-------------------------------------#
num_experiments = 10
rmse_list = []
ece_list = []
nlpd_list = []

# Training parameters
S = 100
epochs = 300
learning_rate = 0.005

for run in range(num_experiments):
    print(f"\n=== Experiment {run+1}/{num_experiments} ===")
    
    # Shuffle and split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=run)
    
    # Scale features and targets to [0,1]
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
    y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))
    
    feature_count = X_train_scaled.shape[1]
    kernel_widths = np.random.uniform(0.1, 2.0, feature_count)
    
    # Initialize RFF layer and base model weights (fresh model each run)
    rff_layer = RFFLayer(S, kernel_widths, feature_count, random_state=run)
    rff_output_dim = 1 + feature_count * S
    w = tf.Variable(tf.zeros((rff_output_dim, 1)), dtype=tf.float32)
    
    def base_prediction(X_input):
        phi = rff_layer(X_input)
        return tf.matmul(phi, w)
    
    def base_loss(X_input, y_true):
        y_pred = base_prediction(X_input)
        mse = tf.reduce_mean(tf.square(y_true - y_pred))
        reg = tf.nn.l2_loss(w)
        return 0.5 * mse + 0.5 * reg

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    # Train the base model
    for epoch in range(1, epochs + 1):
        with tf.GradientTape() as tape:
            loss_val = base_loss(tf.convert_to_tensor(X_train_scaled, dtype=tf.float32),
                                 tf.convert_to_tensor(y_train_scaled, dtype=tf.float32))
        gradients = tape.gradient(loss_val, [w] + rff_layer.trainable_variables)
        optimizer.apply_gradients(zip(gradients, [w] + rff_layer.trainable_variables))
        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Base Loss = {loss_val.numpy():.4f}")
    
    # Compute residuals on training data
    X_train_tf = tf.convert_to_tensor(X_train_scaled, dtype=tf.float32)
    y_train_pred = base_prediction(X_train_tf).numpy()
    residuals_train = y_train_scaled - y_train_pred  # shape (n_samples, 1)
    
    # Train GMR on residuals (concatenate X and residual)
    train_data_gmr = np.hstack((X_train_scaled, residuals_train))
    
    # Select the optimal number of GMM components using BIC
    n_components_range = range(1, 11)
    bic_scores = []
    for n in n_components_range:
        gm = GaussianMixture(n_components=n, covariance_type='full', random_state=run)
        gm.fit(train_data_gmr)
        bic_scores.append(gm.bic(train_data_gmr))
    optimal_n = n_components_range[np.argmin(bic_scores)]
    print("Optimal number of GMM components:", optimal_n)
    
    # Fit the GMR model using the gmr library
    gmr_model = GMM(n_components=optimal_n, random_state=run)
    gmr_model.from_samples(train_data_gmr)
    
    # Predict on the test set
    X_test_tf = tf.convert_to_tensor(X_test_scaled, dtype=tf.float32)
    y_base_test = base_prediction(X_test_tf).numpy()
    
    # Use GMR to predict the residual correction
    input_indices = list(range(feature_count))
    predicted_residuals, predicted_vars = predict_with_covariance(gmr_model, input_indices, X_test_scaled)
    
    # Compute overall predicted standard deviation and enforce a global floor
    min_std = 0
    predicted_std = np.maximum(np.sqrt(predicted_vars), min_std)
    
    # Final prediction: base prediction + residual correction
    y_pred_test = y_base_test.flatten() + predicted_residuals
    
    # Evaluate metrics on scaled targets
    run_rmse = rmse(tf.convert_to_tensor(y_test_scaled, dtype=tf.float32), 
                    tf.convert_to_tensor(y_pred_test.reshape(-1,1), dtype=tf.float32))
    run_ece = ece(y_test_scaled, y_pred_test)
    run_nlpd = nlpd(y_test_scaled, y_pred_test, predicted_std)
    
    print(f"Run {run+1} - RMSE: {run_rmse:.4f}, ECE: {run_ece:.4f}, NLPD: {run_nlpd:.4f}")
    
    rmse_list.append(run_rmse)
    ece_list.append(run_ece)
    nlpd_list.append(run_nlpd)

# Report mean and standard deviation of metrics over experiments
print("\n=== Final Results over 10 Experiments ===")
print(f"Test RMSE: Mean = {np.mean(rmse_list):.4f}, Std = {np.std(rmse_list):.4f}")
print(f"Test ECE:  Mean = {np.mean(ece_list):.4f}, Std = {np.std(ece_list):.4f}")
print(f"Test NLPD: Mean = {np.mean(nlpd_list):.4f}, Std = {np.std(nlpd_list):.4f}")

# Verify that no GPUs are used
print("GPUs available:", tf.config.list_physical_devices('GPU'))



=== Experiment 1/10 ===
Epoch 100: Base Loss = 0.0179
Epoch 200: Base Loss = 0.0179
Epoch 300: Base Loss = 0.0179
Optimal number of GMM components: 5


/tmp/ipykernel_713894/3076141758.py:85: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  comp_means[k] = cond_mean
/tmp/ipykernel_713894/3076141758.py:86: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  comp_vars[k] = cond_var


Run 1 - RMSE: 0.0842, ECE: 0.0242, NLPD: -17.6828

=== Experiment 2/10 ===
Epoch 100: Base Loss = 0.0215
Epoch 200: Base Loss = 0.0215
Epoch 300: Base Loss = 0.0215
Optimal number of GMM components: 4
Run 2 - RMSE: 0.0870, ECE: 0.0136, NLPD: -9.7902

=== Experiment 3/10 ===


/tmp/ipykernel_713894/3076141758.py:85: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  comp_means[k] = cond_mean
/tmp/ipykernel_713894/3076141758.py:86: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  comp_vars[k] = cond_var


Epoch 100: Base Loss = 0.0197
Epoch 200: Base Loss = 0.0197
Epoch 300: Base Loss = 0.0197
Optimal number of GMM components: 5
Run 3 - RMSE: 0.0904, ECE: 0.0277, NLPD: -12.6677

=== Experiment 4/10 ===


...

/tmp/ipykernel_713894/3076141758.py:85: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  comp_means[k] = cond_mean
/tmp/ipykernel_713894/3076141758.py:86: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  comp_vars[k] = cond_var


Epoch 100: Base Loss = 0.0198
Epoch 200: Base Loss = 0.0198
Epoch 300: Base Loss = 0.0198
Optimal number of GMM components: 6


/tmp/ipykernel_713894/3076141758.py:85: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  comp_means[k] = cond_mean
/tmp/ipykernel_713894/3076141758.py:86: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  comp_vars[k] = cond_var


Run 9 - RMSE: 0.0775, ECE: 0.0162, NLPD: -12.5983

=== Experiment 10/10 ===
Epoch 100: Base Loss = 0.0219
Epoch 200: Base Loss = 0.0219
Epoch 300: Base Loss = 0.0219
Optimal number of GMM components: 4
Run 10 - RMSE: 0.1135, ECE: 0.0356, NLPD: -18.9052

=== Final Results over 10 Experiments ===
Test RMSE: Mean = 0.0861, Std = 0.0134
Test ECE:  Mean = 0.0206, Std = 0.0070
Test NLPD: Mean = -17.2591, Std = 9.4235
GPUs available: []


/tmp/ipykernel_713894/3076141758.py:85: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  comp_means[k] = cond_mean
/tmp/ipykernel_713894/3076141758.py:86: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  comp_vars[k] = cond_var
